## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Nesta base de dados, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [10]:
# Criando dados fictícios

import numpy as np
from scipy.signal import stft

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data

array([[[ 1.1135820e+01, -3.3732762e+00,  2.1462999e+01, ...,
          3.0357742e+00, -7.1096511e+00, -9.5652628e+00],
        [ 3.7593274e+00,  1.3652233e+00, -3.9477463e+00, ...,
         -1.4193622e+01,  2.2907005e+01, -7.0980725e+00],
        [-1.0561596e+00,  9.3626366e+00, -1.9367222e+00, ...,
          8.7861493e-02,  6.2952981e+00, -2.8334233e-01],
        [ 5.8598485e+00,  6.5539212e+00, -5.5645032e+00, ...,
         -1.9891384e+01, -2.2424119e+00,  3.9244108e+00]],

       [[-1.1160199e+01, -7.7031106e-01, -2.1216917e+00, ...,
          1.4439607e+01,  1.6906164e+01, -5.1118035e+00],
        [ 1.5013824e+00,  1.2619434e+01,  1.3907468e+01, ...,
          3.2696301e+01, -3.5210538e-01, -6.3649540e+00],
        [-2.0097725e+01,  3.5749493e+00,  7.2583780e+00, ...,
         -1.7456546e+00, -1.0357291e+00,  9.4702941e-01],
        [ 3.4847982e+00,  3.3135300e+00,  2.1765718e+01, ...,
         -1.7350679e+01, -6.6286831e+00,  1.7730516e+00]],

       [[-1.5579151e+00,  2.6120999e

In [11]:
step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


##### Desafio: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).

In [12]:
step = 1
segment = 64
chunks_time = None
chunks_freq = None

while True:
    n_win = int((data.shape[-1] - segment) / step) + 1
    ids = np.arange(n_win) * step

    # Janelas do dado no dominio do tempo
    chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

    # Janelas do dado no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    if chunks_time.shape[2] == chunks_freq.shape[2]:
        break
    else:
        step += 1

print("Step", step)
print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

Step 29
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)
